In [2]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from pyclim_noresm.general_util_funcs import global_avg
from workflow.scripts.utils import (calc_error, calc_abs_change,
                                compute_annual_emission_budget) 
                                    
import matplotlib
import numpy as np
import yaml
import dataframe_image as dfi
from functools import partial

In [3]:
keys = list(snakemake.input.keys())
timeslice = snakemake.params.get('time_slice', slice(5,30))

In [4]:
exp_keys = sorted([k for k in keys if 'exp' in k.split('_')])
ctrl_keys = sorted([k for k in keys if 'ctrl' in k.split('_')])
remainingkeys = list(set(keys)-set(exp_keys+ctrl_keys+ ['areacello']))
areapaths = sorted(snakemake.input['areacello'])

In [5]:
data={}
error={}
for exp_key, ctrl_key in zip(exp_keys,ctrl_keys):
    exp_paths = sorted(snakemake.input[exp_key])
    ctrl_paths = sorted(snakemake.input[ctrl_key])
    vname = xr.open_dataset(exp_paths[0]).variable_id
    if 'emidust' in exp_key.split('_'):
        iterator = zip(exp_paths, ctrl_paths, areapaths)
    else:
        iterator = zip(exp_paths, ctrl_paths)
    data[vname] = {}
    error[vname] = {}
    for paths in iterator:
        
        if 'emidust' in exp_key.split('_'):
            
            ga = xr.open_dataset(paths[2])
            ds_ctrl = xr.open_dataset(paths[1])
            ds_exp = xr.open_dataset(paths[0])
            exp_budget = compute_annual_emission_budget(ds_exp, ga)
            ctrl_budget = compute_annual_emission_budget(ds_ctrl,ga)
            delta_emi = exp_budget-ctrl_budget
            data[vname][ds_ctrl.source_id] = float(delta_emi.values)
        else:
            ds_exp = xr.open_dataset(paths[0]).load()
            ds_ctrl = xr.open_dataset(paths[1]).load()
            diff = calc_abs_change(ds_ctrl, ds_exp, time_slice=timeslice)
            
            diff = global_avg(diff)[vname]
            error[vname][ds_ctrl.source_id] = diff.std().values 
            data[vname][ds_ctrl.source_id] = diff.mean().values 
            

In [6]:
for key in remainingkeys:
    paths = snakemake.input[key]
    for path in paths:
        if path.endswith('yaml'):
            
            with open(path,'r') as f:
                ds = yaml.safe_load(f)
            
            source_id = ds.pop('name')
            for subk in ds:
                vname = "{}_{}".format(key,subk)
                if subk in ['delta_ems'] or vname in ['feedback_Direct_dCdT','feedback_Direct_dT_dEms', 'feedback_Clouds_dCdT','feedback_Clouds_dT_dEms	']:
                    continue
                else:
                    if vname not in data.keys():
                        data[vname] = {}
                        error[vname] = {}
                
                    data[vname][source_id] = np.mean(ds[subk]['data'])
                    error[vname][f'{source_id}'] = float(np.std(ds[subk]['data']))
        else:
            ds = xr.open_dataset(path)
            vname = list(ds.data_vars)[0]
            source_id = ds.source_id
            if vname not in data.keys():
                data[vname] = {}
                error[vname] = {}
            if 'year' in ds.dims:
                ds = ds.rename_dims(year='time')
            erf = global_avg(ds)
            
            data[vname][source_id] = float(erf[vname].mean(dim='time').values)
            error[vname][f'{source_id}'] = float(erf[vname].std(dim='time').values)

In [7]:
df = pd.DataFrame(data, dtype=np.float)
dfe = pd.DataFrame(error, index=df.index,columns=df.columns, dtype=np.float)
df=df.replace([np.inf, -np.inf], np.nan)
dfe = dfe.replace([np.inf, -np.inf], np.nan)
multimodel_mean =  df.mean(axis=0)

std = df.std(axis=0)
dferror = std/np.sqrt(len(df.index))



df.loc['Multi-model',:] = multimodel_mean
dfe.loc['Multi-model',:] = dferror

In [8]:
def greater_than_variability(v,dfe):
    cond = v.abs()>dfe
    return pd.DataFrame(np.where(cond,'font-weight: bold', None),index=v.index,columns=v.columns)
    

In [10]:
forcing_translation = {
    'SWDirectEff': 'SW Fari',
    'LWDirectEff': 'LW Fari',
    'DirectEff': 'Fari',
    'CloudEff': 'Faci',
    'SWCloudEff':'SW Faci',
    'ERFt':'ERF total',
    'LWCloudEff': 'LW Faci',
    'feedback_tot_alpha' : '&alpha; emissions Wm-2 K-1'
}

In [11]:
forcing_componets =['SWDirectEff','LWDirectEff','DirectEff','SWCloudEff','LWCloudEff','CloudEff','ERFt']
dfe_forcing = dfe[forcing_componets].rename(columns=forcing_translation)
df_forcing = df[forcing_componets].rename(columns=forcing_translation)

f = partial(greater_than_variability,dfe=dfe_forcing)


dst=df_forcing.style.apply(f, axis=None)\
.bar(color=['#2717a3','#d95148'], height=50, align='zero',
              width=60, props="width: 100px; border-right: 1px solid black;", axis=0)\
    .format(precision=3, na_rep='')\
    .set_caption('Decomposition of dust radiatve radiative forcings W m-2') .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.5em;'
 }], overwrite=False)
dst

In [12]:
dfi.export(dst,snakemake.output.forcing_table)


In [15]:
diag_translation = {
    'clivi' : 'Ice Water Path g m-2',
    'clt'   : 'Cloud cover (%)',
    'clwvi'   : 'Liquid Water Path g m-2',
    'atmabs': 'Atmoshperic absorption (W m-2)',
    'pr'    : 'Precipitation g m-2 s-1'
    
}

In [16]:
diagnostics_compotents = ['clivi','clt','clwvi','pr']
dfe_diag = dfe[diagnostics_compotents].rename(columns=diag_translation)
df_diag = df[diagnostics_compotents].rename(columns=diag_translation)

f = partial(greater_than_variability,dfe=dfe_diag)


dst=df_diag.style.apply(f, axis=None)\
.bar(color=['#2717a3','#d95148'], height=50, align='zero',
              width=60, props="width: 100px; border-right: 1px solid black;", axis=0)\
    .format(precision=3, na_rep='', 
            formatter={
                'Precipitation g m-2 s-1': lambda x: "{:.2e}".format(x*1e3),
                'Liquid Water Path g m-2': lambda x: "{:.3f}".format(x*1e3),
                'Ice Water Path g m-2': lambda x: "{:.3f}".format(x*1e3)
            }
           )\
    .set_caption('Absolute change between control and 2x-dust') .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.5em;'
 }], overwrite=False)
dst

In [14]:
dfi.export(dst,snakemake.output.diagnostics_table_path)

In [15]:
feedback_translation = {
    'feedback_tot_alpha' : '&alpha; emissions Wm-2 K-1',
    'feedback_tot_dCdT'  : '&Delta; emission /&Delta;T Tgyr-1 K-1' 
}

In [16]:
feedback_components = ['feedback_tot_dCdT','feedback_tot_alpha']
dfe_diag = dfe[feedback_components].rename(columns=feedback_translation)
df_diag = df[feedback_components].rename(columns=feedback_translation)

f = partial(greater_than_variability,dfe=dfe_diag)


dst=df_diag.style.apply(f, axis=None)\
.bar(color=['#2717a3','#d95148'], height=50, align='zero',
              width=60, props="width: 100px; border-right: 1px solid black;", axis=0)\
    .format(precision=3, na_rep='', formatter={
    '&Delta; emission /&Delta;T Tgyr-1 K-1': lambda x: "{:.1f}".format(x),
}
           )\
    .set_caption('Emission emission change per temperature change and Feedback parameter') .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.2em;'
 }], overwrite=False)
dst

In [17]:
dfi.export(dst,snakemake.output.feedback_table)

In [18]:
if snakemake.output.outpath.endswith('csv'):
    df.to_csv(snakemake.output.outpath)
elif snakemake.output.outpath.endswith('tex'):
    col_format = ['m{5em}'] + ['m{1.5cm}' for i in range(len(df.columns))]
    col_format = ''.join(col_format)
    with open(snakemake.output.outpath, 'w') as f:
        df.to_latex(buf=f,  na_rep='', col_space=3, longtable=False, column_format= col_format)